In [37]:
import numpy
import time
import pickle
from __future__ import print_function

In [2]:
def load_file(filepath):
    
    with open(filepath, 'r') as f:
        file = f.readlines()
    return file

src_file = load_file('data/file.en')
trg_file = load_file('data/file.de')
aligned_file = load_file('data/file.aligned')

In [3]:
def extract(trg_start, trg_end, src_start, src_end, alignments, trg_aligned, trg_len):    
 
    if trg_end < 0 :
        return []
    
    for trg_word, src_word  in alignments:
        if ((trg_start <= trg_word <= trg_end) and 
           (src_word < src_start or src_word > src_end)):  
            return []
        
    E = []
    trg_s = trg_start
    
    while True:
        trg_e = trg_end
        while True:
            E.append( (tuple(trg_words[trg_s:trg_e+1]) , tuple(src_words[src_start:src_end+1])) )
            
            trg_e += 1
            
            if trg_e in trg_aligned or trg_e == len(trg_words):
                break 
    
        trg_s -= 1
        
        if trg_s in trg_aligned or trg_s < 0:
            break
    return E


def extract_phrases(src_words,trg_words,alignments):
    
    trg_aligned = set()
    for (trg,_) in alignments:
        trg_aligned.add(trg)
    
    
    extracted_phrases = []
    for src_start in range(len(src_words)):
        for src_end in range(src_start, len(src_words)):

            trg_start = len(trg_words)
            trg_end = 0
            for  (trg,src) in alignments: 
                if src_start <= src <= src_end:
                    trg_start = min(trg, trg_start)
                    trg_end = max(trg, trg_end)

            phrase_pairs = extract(trg_start, trg_end, src_start, src_end, alignments, trg_aligned, len(trg_words))
            if (phrase_pairs):
                extracted_phrases.extend(phrase_pairs)
            
    return extracted_phrases

In [4]:
def add_freq_dict(dictionary, item):
    if item in dictionary:
        dictionary[item] += 1
    else:
        dictionary[item] = 1

src_words = "michael assumes that he will stay in the house".split()
trg_words = "michael geht davon aus , dass er im haus bleibt".split()
alignments = [(0,0), (1,1), (2,1), (3,1), (5,2), (6,3), (9,4), (9,5), (7,6), (7,7), (8,8)]
 
    
    
BP = extract_phrases(src_words, trg_words, alignments)
    
freq_pairs = dict()
freq_src = dict()
freq_trg = dict()
    
for pair in BP:
    add_freq_dict(freq_pairs,pair)
    add_freq_dict(freq_trg, pair[0])
    add_freq_dict(freq_src, pair[1])

In [28]:
freq_pairs = dict()
freq_src = dict()
freq_trg = dict()
    
for pair in BP:
    add_freq_dict(freq_pairs,pair)
    add_freq_dict(freq_trg, pair[0])
    add_freq_dict(freq_src, pair[1])
    

start_time = time.time()
    
for i in range(len(src_file)):
    src_words = src_file[i].split()
    trg_words = trg_file[i].split()        
    alignments = [[int(a) for a in alignment.split('-')] for alignment in aligned_file[i].split()]
    
    BP = extract_phrases(src_words, trg_words, alignments)
    
    for pair in BP:
        add_freq_dict(freq_pairs,pair)
        add_freq_dict(freq_trg, pair[0])
        add_freq_dict(freq_src, pair[1])
        
    print(' \r%d / %d (%d s)'%(i+1,len(src_file), time.time() - start_time), end = '')
    
print('Done!!!\nTotal duration: %d'%(time.time() - start_time))

50000 / 50000 (585 s)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [38]:
with open("data/freq_src", 'wb') as file:
    pickle.dump(freq_src, file)

with open("data/freq_trg", 'wb') as file:
    pickle.dump(freq_trg, file)

with open("data/freq_pairs", 'wb') as file:
    pickle.dump(freq_pairs, file)
        

In [43]:
import operator
sorted_freq_src = sorted(freq_src.items(), key=operator.itemgetter(1), reverse = True)
sorted_freq_trg = sorted(freq_trg.items(), key=operator.itemgetter(1), reverse = True)
sorted_freq_pairs = sorted(freq_pairs.items(), key=operator.itemgetter(1), reverse = True)